1. Run ```docker run -d -p 2222:22 --name mysshserver rastasheep/ubuntu-sshd``` to setup a local linux container with ssh enabled on port 2222
2. Make sure you have paramiko installed: ```pip install paramiko```


NOTE: This implementation is slightly different from the ShellHandler actually used in the react agent (the other implementation uses private key file name and doesn't require password).

This implementation is copied from [stackoverflow](https://stackoverflow.com/a/36948840) with slight changes (echo command is run together with the user command).

In [33]:
import paramiko
import re

class ShellHandler:

    def __init__(self, hostname, username, password):
        self.ssh = paramiko.SSHClient()
        self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.ssh.connect(hostname, username=username, password=password, port=2222)

        channel = self.ssh.invoke_shell()
        self.stdin = channel.makefile('wb')
        self.stdout = channel.makefile('r')

    def __del__(self):
        self.ssh.close()
        
    def pretty_execute(self, cmd):
        shin, shout, sherr = self.execute(cmd)
        output = ''.join(shout)
        error = ''.join(sherr)
        return output if len(error) == 0 else f"""{output}\nError: {error}"""

    def execute(self, cmd):
        """

        :param cmd: the command to be executed on the remote computer
        :examples:  execute('ls')
                    execute('finger')
                    execute('cd folder_name')
        """
        cmd = cmd.strip('\n')
        finish = 'end of stdOUT buffer. finished with exit status'
        echo_cmd = "echo {} $?".format(finish)
        self.stdin.write(cmd + ' ; ' + echo_cmd + '\n')
        shin = self.stdin
        self.stdin.flush()

        shout = []
        sherr = []
        exit_status = 0
        for line in self.stdout:
            if echo_cmd in str(line):
                # up for now filled with shell junk from stdin
                shout = []
            elif finish in str(line):
                # our finish command ends with the exit status
                exit_status = int(str(line).rsplit(maxsplit=1)[1])
                index = str(line).find(finish)
                shout.append(str(line)[:index]) 
                if exit_status:
                    # stderr is combined with stdout.
                    # thus, swap sherr with shout in a case of failure.
                    sherr = list(shout)
                    shout = []
                break
            else:
                # get rid of 'coloring and formatting' special characters
                formatted_line = re.compile(r'(\x9B|\x1B\[)[0-?]*[ -/]*[@-~]').sub('', line).replace('\b', '').replace('\r', '')
                shout.append(formatted_line)

        return shin, shout, sherr

In [34]:
shell_handler = ShellHandler(
    hostname="localhost",
    username="root",
    password="root",    
)

In [36]:
print(shell_handler.pretty_execute("ls"))

admin@ip-172-31-36-211:~$ ls ; echo end of stdOUT buffer. finished with exit status $?

resources

end of stdOUT buffer. finished with exit status 0

0
resources



In [37]:
print(shell_handler.pretty_execute("pwd"))

admin@ip-172-31-36-211:~$ pwd ; echo end of stdOUT buffer. finished with exit status $?

/home/admin

end of stdOUT buffer. finished with exit status 0

0
/home/admin



In [38]:
print(shell_handler.pretty_execute("cd .."))

admin@ip-172-31-36-211:~$ cd resources ; echo end of stdOUT buffer. finished with exit status $?

end of stdOUT buffer. finished with exit status 0

9



In [39]:
print(shell_handler.pretty_execute("ls"))

admin@ip-172-31-36-211:~/resources$ ls ; echo end of stdOUT buffer. finished with exit status $?

A  C  D  start.txt

end of stdOUT buffer. finished with exit status 0

0
A  C  D  start.txt



In [40]:
print(shell_handler.pretty_execute("pinkkk"))

admin@ip-172-31-36-211:~/resources$ pinkkk ; echo end of stdOUT buffer. finished with exit status $?

-bash: pinkkk: command not found

end of stdOUT buffer. finished with exit status 127

0

Error: -bash: pinkkk: command not found



In [41]:
print(shell_handler.pretty_execute("python"))

admin@ip-172-31-36-211:~/resources$ python ; echo end of stdOUT buffer. finished with exit status $?

-bash: python: command not found

end of stdOUT buffer. finished with exit status 127

0

Error: -bash: python: command not found



In [42]:
print(shell_handler.pretty_execute("apt update"))

admin@ip-172-31-36-211:~/resources$ apt update ; echo end of stdOUT buffer. finished with exit status $?

Reading package lists... Done

E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)

E: Unable to lock directory /var/lib/apt/lists/

W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)

W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)

end of stdOUT buffer. finished with exit status 100

0

Error: Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)

